In [1]:
import sys

src_dir = '../src'

if src_dir not in sys.path:
    sys.path.append(src_dir)

## Training of simple dense net

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from classifier.dataloader.dataloader import create_dataloaders
from classifier.models.simple_net import NeuralNetwork

In [21]:
torch.manual_seed(0)

In [22]:
data_raw_path = '../data/data_raw.csv'
batch_size = 32
max_features = 1e4
train_ratio = .7
val_ratio = .15
shuffle = True

(

    train_loader, 
    val_loader, 
    test_loader

) = create_dataloaders(

    csv_path=data_raw_path,
    batch_size=batch_size,
    max_features=int(max_features),
    train_ratio=train_ratio,
    val_ratio=val_ratio,
    shuffle=shuffle

)

In [23]:
learning_rate = 0.001
epochs = 5

input_size = int(max_features)
hidden_sizes = [128, 64]

model = NeuralNetwork(
    input_size=input_size,
    hidden_sizes=hidden_sizes
)

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
from classifier.training_loop import train, evaluate

In [26]:
train(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    data_loader=train_loader, 
    epochs=5
)

Epoch 1/5:   0%|          | 0/219 [00:00<?, ?batch/s]

Epoch 2/5:   0%|          | 0/219 [00:00<?, ?batch/s]

Epoch 3/5:   0%|          | 0/219 [00:00<?, ?batch/s]

Epoch 4/5:   0%|          | 0/219 [00:00<?, ?batch/s]

Epoch 5/5:   0%|          | 0/219 [00:00<?, ?batch/s]

In [27]:
average_loss, accuracy = evaluate(
    model=model,
    criterion=criterion,
    data_loader=test_loader
)

Average Loss: 1.8722, Accuracy: 35.00%
